# JSON Schema

A JSON schema declares the expected format of a JSON document. It is a popular method for documenting your interfaces that use JSON as communication format.

In [18]:
from typing import Any
from jsonschema import validate

def validate_q(instance: Any, schema: dict):
    try:
        validate(instance=instance, schema=schema)
    except Exception as e:
        print(e)

A JSON Schema is a standard used to define and validate the structure and data types of a JSON document (the instance). It is itself a JSON document where the structure and constraints for each **property** are described using predefined **schema keywords** (e.g., type, properties).

For examples chema:

In [19]:
schema = {"type": "number"}

Specifies the the root level of the JSON should be a number, e.g. JSON by itself a number.

In [23]:
validate_q(schema=schema, instance=10)

The following cell specifies that the root object is a mapping, where the key `"wow"` should be a string. 

In [27]:
schema = {
    "type": "object",
    "wow": {
        "type": "string"
    }
}
validate_q(instance={"wow": "hello"}, schema=schema)

## Type

The `type` keyword is used to specify the type of the attribute.

Check more in [Type-specified Keywords](https://json-schema.org/understanding-json-schema/reference/type).

---

The following cell indicates that the root element of the JSON should be a string.

In [ ]:
schema = {"type": "string"}

Specifying a string works fine.

In [30]:
validate_q(schema=schema, instance="some string")

However, any other object, such as number, will result in the corresponding error.

In [28]:
validate_q(schema=schema, instance=10)

10 is not of type 'object'

Failed validating 'type' in schema:
    {'type': 'object', 'wow': {'type': 'string'}}

On instance:
    10
